# Alzheimer's Disease and Frontotemporal Dementia Prediction


- Author: Elmo Chavez
- Date: 19-Jul-2023

**Description**

> [...]


## Read the Data


### Import Libraries


In [2]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

### Reading Datasets


In [3]:
# Filenames
#filename_participants = 'participants_selected.csv'
filename_epochs_fp1 = 'epoch_psd_features.csv'
filename_epochs_all = 'epoch_psd_features_all_channels.csv'
filename_epochs_bnds_fp1 = 'epoch_bands_psd_features.csv'
filename_epochs_bdns_all = 'epoch_bands_psd_features_all_channels.csv'

# path
path = os.getcwd()

# Participants info
#df_participants = pd.read_csv(path+'/Feature_Extraction/'+filename_participants)

# Epochs based feature extraction
df_epochs_fp1 = pd.read_csv(path+'/Feature_Extraction/'+filename_epochs_fp1)
df_epochs_all = pd.read_csv(path+'/Feature_Extraction/'+filename_epochs_all)

# Epochs and Frequency Bands feature extraction
df_epochs_bands_fp1 = pd.read_csv(path+'/Feature_Extraction/'+filename_epochs_bnds_fp1)
df_epochs_bands_all = pd.read_csv(path+'/Feature_Extraction/'+filename_epochs_bdns_all)

# Shape of the data
#print('Participants info:',df_participants.shape)
print('Epochs base Feature Extraction - FP1:',df_epochs_fp1.shape)
print('Epochs base Feature Extraction - All Channels:',df_epochs_all.shape)
print('Epochs base with Freq. Bands Feature Extraction - FP1:',df_epochs_bands_fp1.shape)
print('Epochs base with Freq. Bands Feature Extraction - All Channels:',df_epochs_bands_all.shape)

Epochs base Feature Extraction - FP1: (714, 11)
Epochs base Feature Extraction - All Channels: (714, 11)
Epochs base with Freq. Bands Feature Extraction - FP1: (714, 31)
Epochs base with Freq. Bands Feature Extraction - All Channels: (714, 31)


## Train Test Split


#### Epoch based FP1


In [4]:
df_epochs_fp1.head()

,subject_id,age,gender,epoch_id,total_channels,total_power,avg_power,relative_power,peak_freq,spectral_entropy,group
0,sub-002,78,1,0,19.0,8.138737e-07,3.255495e-08,0.480204,1.953125,0.000013,0
1,sub-002,78,1,1,19.0,6.457802e-07,2.583121e-08,0.373726,1.953125,0.000010,0
2,sub-002,78,1,2,19.0,7.862074e-07,3.144830e-08,0.397717,1.953125,0.000013,0
3,sub-002,78,1,3,19.0,8.496011e-07,3.398405e-08,0.458611,1.953125,0.000013,0
4,sub-002,78,1,4,19.0,7.503180e-07,3.001272e-08,0.402417,1.953125,0.000012,0


In [5]:
df_epochs_fp1_grp = df_epochs_fp1.groupby('subject_id').mean().reset_index()
print(df_epochs_fp1_grp.shape)
df_epochs_fp1_grp.head()

(51, 11)


,subject_id,age,gender,epoch_id,total_channels,total_power,avg_power,relative_power,peak_freq,spectral_entropy,group
0,sub-002,78.0,1.0,6.5,19.0,8.747903e-07,3.499161e-08,0.440120,1.953125,0.000014,0.0
1,sub-004,67.0,1.0,6.5,19.0,1.032087e-06,4.128349e-08,0.459969,1.953125,0.000016,0.0
2,sub-005,70.0,0.0,6.5,19.0,1.147346e-06,4.589383e-08,0.461923,1.953125,0.000017,0.0
3,sub-006,61.0,1.0,6.5,19.0,1.349809e-06,5.399235e-08,0.489254,1.953125,0.000020,0.0
4,sub-007,79.0,1.0,6.5,19.0,9.680973e-07,3.872389e-08,0.437120,1.953125,0.000015,0.0


In [6]:
cols_to_drop = ['subject_id','group']

X = df_epochs_fp1_grp.drop(columns=cols_to_drop, axis=1)
y = df_epochs_fp1_grp['group']

X_train_fp1, X_test_fp1, y_train_fp1, y_test_fp1 = train_test_split(X, y, test_size=0.25, random_state=42)

print('Train',len(X_train_fp1))
print('Test',len(X_test_fp1))

Train 38
Test 13


#### Epochs based All Channels


In [7]:
df_epochs_all.head()

,subject_id,age,gender,epoch_id,total_channels,total_power,avg_power,relative_power,peak_freq,spectral_entropy,group
0,sub-002,78,1,0,19.0,8.346040e-07,3.338416e-08,0.478456,1.953125,0.000013,0
1,sub-002,78,1,1,19.0,6.761889e-07,2.704755e-08,0.374997,1.953125,0.000011,0
2,sub-002,78,1,2,19.0,8.086165e-07,3.234466e-08,0.394570,1.953125,0.000013,0
3,sub-002,78,1,3,19.0,8.440792e-07,3.376317e-08,0.447835,1.953125,0.000013,0
4,sub-002,78,1,4,19.0,7.673421e-07,3.069369e-08,0.416419,1.953125,0.000012,0


In [8]:
df_epochs_all_grp = df_epochs_fp1.groupby('subject_id').mean().reset_index()

X = df_epochs_all_grp.drop(columns=cols_to_drop, axis=1)
y = df_epochs_fp1_grp['group']

X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X, y, test_size=0.25, random_state=42)

print('Train',len(X_train_fp1))
print('Test',len(X_test_fp1))

Train 38
Test 13


#### Epochs based with Frequency Bands FP1


In [9]:
df_epochs_bands_fp1.head()

,epoch_id,total_channels,delta_total_power,delta_avg_power,delta_relative_power,delta_peak_freq,delta_spectral_entropy,theta_total_power,theta_avg_power,theta_relative_power,...,beta_spectral_entropy,gamma_total_power,gamma_avg_power,gamma_relative_power,gamma_peak_freq,gamma_spectral_entropy,subject_id,age,gender,group
0,0,19.0,5.356440e-07,2.678220e-07,0.316042,1.953125,0.000008,1.703684e-08,8.518421e-09,0.010052,...,0.000001,1.292570e-07,1.615713e-08,0.076265,44.921875,0.000002,sub-002,78,1,0
1,1,19.0,3.903346e-07,1.951673e-07,0.225894,1.953125,0.000006,2.239278e-08,1.119639e-08,0.012959,...,0.000002,1.108229e-07,1.385286e-08,0.064135,35.156250,0.000002,sub-002,78,1,0
2,2,19.0,5.090068e-07,2.545034e-07,0.257490,1.953125,0.000008,3.471327e-08,1.735663e-08,0.017560,...,0.000002,8.779028e-08,1.097378e-08,0.044410,37.109375,0.000002,sub-002,78,1,0
3,3,19.0,5.687169e-07,2.843584e-07,0.306991,1.953125,0.000008,2.759734e-08,1.379867e-08,0.014897,...,0.000001,1.364043e-07,1.705053e-08,0.073631,39.062500,0.000002,sub-002,78,1,0
4,4,19.0,5.352662e-07,2.676331e-07,0.287079,1.953125,0.000008,2.820804e-08,1.410402e-08,0.015129,...,0.000001,9.871103e-08,1.233888e-08,0.052942,42.968750,0.000002,sub-002,78,1,0


In [10]:
df_epochs_bands_fp1_grp = df_epochs_bands_fp1.groupby('subject_id').mean().reset_index()

X = df_epochs_bands_fp1_grp.drop(columns=cols_to_drop, axis=1)
y = df_epochs_bands_fp1_grp['group']

X_train_fbnds_fp1, X_test_fbnds_fp1, y_train_fbnds_fp1, y_test_fbnds_fp1 = train_test_split(X, y, test_size=0.25, random_state=42)

print('Train',len(X_train_fbnds_fp1))
print('Test',len(X_test_fbnds_fp1))

Train 38
Test 13


#### Epochs based with Frequency Bands All Channels


In [25]:
df_epochs_bands_all.head()

,epoch_id,total_channels,delta_total_power,delta_avg_power,delta_relative_power,delta_peak_freq,delta_spectral_entropy,theta_total_power,theta_avg_power,theta_relative_power,...,beta_spectral_entropy,gamma_total_power,gamma_avg_power,gamma_relative_power,gamma_peak_freq,gamma_spectral_entropy,subject_id,age,gender,group
0,0,19.0,5.607184e-07,2.803592e-07,0.321419,1.953125,0.000008,1.780230e-08,8.901150e-09,0.010167,...,0.000001,1.402790e-07,1.753488e-08,0.080389,41.118421,0.000002,sub-002,78,1,0
1,1,19.0,4.036695e-07,2.018348e-07,0.224124,1.953125,0.000006,2.129087e-08,1.064544e-08,0.011726,...,0.000002,1.166759e-07,1.458449e-08,0.064537,35.361842,0.000002,sub-002,78,1,0
2,2,19.0,5.249767e-07,2.624883e-07,0.256317,1.953125,0.000008,3.140269e-08,1.570135e-08,0.015305,...,0.000002,1.045114e-07,1.306392e-08,0.050853,37.109375,0.000002,sub-002,78,1,0
3,3,19.0,5.627172e-07,2.813586e-07,0.298903,1.953125,0.000008,2.816719e-08,1.408359e-08,0.014952,...,0.000002,1.184824e-07,1.481030e-08,0.062715,37.828947,0.000002,sub-002,78,1,0
4,4,19.0,5.368642e-07,2.684321e-07,0.291363,1.953125,0.000008,3.484860e-08,1.742430e-08,0.018918,...,0.000001,1.033295e-07,1.291619e-08,0.056063,39.268092,0.000002,sub-002,78,1,0


In [26]:
df_epochs_bands_all_grp = df_epochs_bands_all.groupby('subject_id').mean().reset_index()

X = df_epochs_bands_all_grp.drop(columns=cols_to_drop, axis=1)
y = df_epochs_bands_all_grp['group']

X_train_fbnds_all, X_test_fbnds_all, y_train_fbnds_all, y_test_fbnds_all = train_test_split(X, y, test_size=0.25, random_state=42)

print('Train',len(X_train_fbnds_all))
print('Test',len(X_test_fbnds_all))

Train 38
Test 13


## Predictions


### Epochs Based - FP1 Channel


XGBoost


In [11]:
xgb_classifier = xgb.XGBClassifier()
xgb_classifier.fit(X_train_fp1, y_train_fp1)

xgb_predict = xgb_classifier.predict(X_test_fp1)

# Accuracy
accuracy = accuracy_score(y_test_fp1, xgb_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 61.54%


AdaBoost


In [12]:
adaboost_classifier = AdaBoostClassifier()
adaboost_classifier.fit(X_train_fp1, y_train_fp1)

adaboost_predict = adaboost_classifier.predict(X_test_fp1)

# Accuracy
accuracy = accuracy_score(y_test_fp1, adaboost_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 30.77%


Random Forest


In [13]:
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_fp1, y_train_fp1)

rf_predict = rf_classifier.predict(X_test_fp1)

# Accuracy
accuracy = accuracy_score(y_test_fp1, rf_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 46.15%


Support Vector Classifier


In [14]:
svc_classifier = SVC()
svc_classifier.fit(X_train_fp1, y_train_fp1)

svc_predict = svc_classifier.predict(X_test_fp1)

# Accuracy
accuracy = accuracy_score(y_test_fp1, svc_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 69.23%


### Epochs Based - All Channels


XGBoost


In [15]:
xgb_classifier = xgb.XGBClassifier()
xgb_classifier.fit(X_train_all, y_train_all)

xgb_predict = xgb_classifier.predict(X_test_all)

# Accuracy
accuracy = accuracy_score(y_test_all, xgb_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 61.54%


AdaBoost


In [16]:
adaboost_classifier = AdaBoostClassifier()
adaboost_classifier.fit(X_train_all, y_train_all)

adaboost_predict = adaboost_classifier.predict(X_test_all)

# Accuracy
accuracy = accuracy_score(y_test_all, adaboost_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 30.77%


Random Forest


In [17]:
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_all, y_train_all)

rf_predict = rf_classifier.predict(X_test_all)

# Accuracy
accuracy = accuracy_score(y_test_all, rf_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 38.46%


Support Vector Classifier


In [18]:
svc_classifier = SVC()
svc_classifier.fit(X_train_all, y_train_all)

svc_predict = svc_classifier.predict(X_test_all)

# Accuracy
accuracy = accuracy_score(y_test_all, svc_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 69.23%


### Epochs Based with Frequency Bands - FP1 Channel


XGBoost


In [20]:
xgb_classifier = xgb.XGBClassifier()
xgb_classifier.fit(X_train_fbnds_fp1, y_train_fbnds_fp1)

xgb_predict = xgb_classifier.predict(X_test_fbnds_fp1)

# Accuracy
accuracy = accuracy_score(y_test_fbnds_fp1, xgb_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 46.15%


AdaBoost


In [21]:
adaboost_classifier = AdaBoostClassifier()
adaboost_classifier.fit(X_train_fbnds_fp1, y_train_fbnds_fp1)

adaboost_predict = adaboost_classifier.predict(X_test_fbnds_fp1)

# Accuracy
accuracy = accuracy_score(y_test_fbnds_fp1, adaboost_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 46.15%


Random Forest


In [22]:
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_fbnds_fp1, y_train_fbnds_fp1)

rf_predict = rf_classifier.predict(X_test_fbnds_fp1)

# Accuracy
accuracy = accuracy_score(y_test_fbnds_fp1, rf_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 53.85%


Support Vector Classifier


In [23]:
svc_classifier = SVC()
svc_classifier.fit(X_train_fbnds_fp1, y_train_fbnds_fp1)

svc_predict = svc_classifier.predict(X_test_fbnds_fp1)

# Accuracy
accuracy = accuracy_score(y_test_fbnds_fp1, svc_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 69.23%


### Epochs Based with Frequency Bands - All Channel


XGBoost


In [27]:
xgb_classifier = xgb.XGBClassifier()
xgb_classifier.fit(X_train_fbnds_all, y_train_fbnds_all)

xgb_predict = xgb_classifier.predict(X_test_fbnds_all)

# Accuracy
accuracy = accuracy_score(y_test_fbnds_all, xgb_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 69.23%


AdaBoost


In [28]:
adaboost_classifier = AdaBoostClassifier()
adaboost_classifier.fit(X_train_fbnds_all, y_train_fbnds_all)

adaboost_predict = adaboost_classifier.predict(X_test_fbnds_all)

# Accuracy
accuracy = accuracy_score(y_test_fbnds_all, adaboost_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 38.46%


Random Forest


In [29]:
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_fbnds_all, y_train_fbnds_all)

rf_predict = rf_classifier.predict(X_test_fbnds_all)

# Accuracy
accuracy = accuracy_score(y_test_fbnds_all, rf_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 76.92%


Support Vector Classifier


In [30]:
svc_classifier = SVC()
svc_classifier.fit(X_train_fbnds_all, y_train_fbnds_all)

svc_predict = svc_classifier.predict(X_test_fbnds_all)

# Accuracy
accuracy = accuracy_score(y_test_fbnds_all, svc_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 69.23%
